In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')

In [2]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

In [3]:
# parameters
customer_id = '5'
formatted_attribute = 'Weight'
customer_name='%bedbathandbeyond%'
buckets = """Multiple Buckets"""
strategy_version_input=668
attribute_id_input=2674


regex_pattern = r"(?i)(?i)(weighs\s?\d*\.?\d*\s?lb)|(?i)(weighs\s?\d*\s?lb)|(Weighs approximately \d* lb)|()"      

In [4]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [5]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [6]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
print(len(df))
df = df[df['bucket_id'].isin(buckets)]
len(df)

694


542

In [7]:
err

NameError: name 'err' is not defined

In [ ]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
x=df[df['matches'].astype(str)!='[]']
print(len(x))

In [ ]:
x['matches']= x['matches'].apply(
    lambda x: sorted(x, key=natural_key))
x.head(3)

In [ ]:
replacement = {
    "Weighs": "",
    'weighs': "",
    'approximately': "",
}

In [ ]:
# Replacement
x['matches']= x['matches'].apply(
    lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)
matches=x
matches.head(3)

In [ ]:
na=df[df['matches'].astype(str)=='[]']
na.head(1)
match_na=na

In [ ]:
#import regex as re
matches[curation_col]=matches['matches'].astype(str)
matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
matches.head(3)

# matches

In [ ]:
matches['matches']=matches['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\'",'',str(measure)))
matches['matches']=matches['matches'].apply(lambda measure: re.sub(r'\\|\"|\,','',str(measure)))
matches.head(3)

In [ ]:
match_na['matches']=match_na['matches'].astype(str).apply(lambda x: 'n/a')
match_na[curation_col]=match_na['matches']
match_na.head(1)

In [ ]:
match_na[curation_col] = r'n/a'

In [ ]:
match_na.to_csv(f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/Bed Bath & Beyond/_Ready For Upload/Bed Bath & Beyond - {attribute} - na upload.csv',index=False)

In [ ]:
new_matches=match_na[['external_id', curation_col]]
new_matches

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/- {attribute}-weight_N-A- match upload_Shawn.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matchez)